*This analysis is preliminary and based on a toy dataset (n=2), intended solely to validate the inference and analysis pipeline. No performance conclusions are drawn.*

In [ ]:
import os, sys

PROJECT_ROOT = "/content/nlp-finetuning-bert"
os.chdir(PROJECT_ROOT)
sys.path.append(PROJECT_ROOT)

print("CWD:", os.getcwd())
assert os.path.exists("data/predictions_with_analysis.csv"), "Brak data/predictions_with_analysis.csv"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import (
    classification_report,
    confusion_matrix
)

In [ ]:
df = pd.read_csv("data/predictions_with_analysis.csv")

df.head(), df.shape

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df["label"].value_counts()

In [ ]:
df["label_thresholded"].value_counts()

In [ ]:
print(
    classification_report(
        df["label"],
        df["label_thresholded"],
        labels=["NEGATIVE", "POSITIVE"],
        zero_division=0
    )
)


confusion_matrix(df["label"], df["label_thresholded"])

At the current stage, the model exhibits a tendency to route low-confidence negative samples into the UNCERTAIN category under a conservative threshold (0.7), which is expected behavior given the design of the inference logic.

In [90]:
!git add notebooks/error_analysis.ipynb
!git commit -m "Clean the code"
!git push

[main ea465d3] Add preliminary error analysis (pipeline validation)
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite notebooks/error_analysis.ipynb (98%)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 742 bytes | 742.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
remote: This repository moved. Please use the new location:
remote:   https://github.com/Masesun/nlp-finetuning-bert.git
To https://github.com/masesun/nlp-finetuning-bert.git
   b9e7baa..ea465d3  main -> main


In [91]:
!cp "/content/drive/MyDrive/Colab Notebooks/nlp-finetuning-bert.ipynb" \
   "/content/nlp-finetuning-bert/notebooks/error_analysis.ipynb"